# Parse Supplier Country % from Earnings Call Transcripts

This notebook demonstrates how to use LSEG Transcripts with OpenAI's GPT-4.5 model to parse out Supplier Country % relationships discussed in a company's Earnings Call, writing the results to an Excel file.  You can run this notebook as-is, with your own LSEG and OpenAI credentials.  Or you an modify it slightly to use another GenAI model.

### Credentials
Swap in your own LSEG Transcripts and Azure OpenAI credenitals

In [ ]:
# Transcripts server details
hostname = "ftp-setranscripts.lseg.com"
username = "" # Add your username here
password = "" # Add your password here
directory = "/TRANSCRIPT/XML_Add_IDs/Current"

# OpenAI details
ai_key = "" # Add your OpenAI API key here
ai_endpoint = "" # Add your OpenAI endpoint here (e.g. https://some_address.openai.azure.com)
ai_model = "" # Add your OpenAI model here (e.g. gpt-4.5-preview_2025-02-27)
ai_version = "" # Add your OpenAI API version here (e.g. 2025-01-01-preview)


### Notebook Parameters

Some parameters for for downloading from the Transcripts SFTP and using OpenAI.  You can reduce the max_files and max_companies to a smaller number as you're experimenting, then use larger numbers when you're confident with code for bigger batch processing.

You can also toggle the SFTP download off, if you've already downloaded Transcripts recently and don't want to repeat that step for the same files again.  In case you want to focus on the GenAI portion of code.

In [2]:
# SFTP download options
download_from_sftp = True                  # If you've already downloaded files recently, set this to False so you can focus on the GenAI processing
local_directory = "transcripts_xml"        # Local directory to store downloaded XML files
lookback_days = 7
max_files = 3000

# GenAI Processing options
only_US = True                              # If True, only process US companies
genai_output_dir = "supplier_countries"     # Local directory for Excel file with GenAI results
max_companies = 500
max_attempts = 1                            # Attempts for OpenAI extraction, since sometimes you might not get a filled out response so you can retry

### ===============================
### === NO NEED TO CHANGE CODE BELOW ===
### ===============================

In [3]:
from datetime import datetime, timedelta
import json
from openai import AzureOpenAI
import os
import pandas as pd
import pysftp
import re
import xml.etree.ElementTree as ET

In [4]:
# Show all columns in DataFrame
pd.set_option('display.max_columns', None)

### Download recent transcript files from SFTP

The XML files will be stored locally in the directory specificed by local_directory.  This code example is focused on recent history, so lookback_days is from present.  The max_files parameter will cap how many XML files you download.

In [5]:
# Create the local directory if it doesn't exist
os.makedirs(local_directory, exist_ok=True)

In [6]:
if download_from_sftp:
    # Connect to the SFTP server
    print("Connecting to SFTP server...")

    # Disable hostkey verification
    cnopts = pysftp.CnOpts()
    cnopts.hostkeys = None  
    
    # Establish the connection
    with pysftp.Connection(host=hostname, username=username, password=password, cnopts=cnopts) as sftp:
        print("Connection successfully established!")
        
        # Change to the target directory
        sftp.cwd(directory)
        
        # Get the list of files in the directory
        files = sftp.listdir_attr()
        
        # Filter files created in the last X days
        x_days_ago = datetime.now() - timedelta(days=lookback_days)
        recent_files = [(file.filename, datetime.fromtimestamp(file.st_mtime)) for file in files if datetime.fromtimestamp(file.st_mtime) > x_days_ago]
        
        # Create a pandas DataFrame
        df_recent_files = pd.DataFrame(recent_files, columns=['Filename', 'CreationDate'])

        # Download the most recent files
        for index, row in df_recent_files.head(max_files).iterrows():
            remote_file = row['Filename']
            local_file = os.path.join(local_directory, remote_file)
            sftp.get(remote_file, local_file)

        print(f"Downloaded the most recent files into the '{local_directory}' directory.")

Connecting to SFTP server...


C:\Users\abaron\AppData\Roaming\Python\Python312\site-packages\pysftp\__init__.py:61: UserWarning: Failed to load HostKeys from C:\Users\abaron\.ssh\known_hosts.  You will need to explicitly load HostKeys (cnopts.hostkeys.load(filename)) or disableHostKey checking (cnopts.hostkeys = None).
  warnings.warn(wmsg, UserWarning)


Connection successfully established!
Downloaded the most recent files into the 'transcripts_xml' directory.


### Read downloaded transcript files from local directory

This bit of code can run and re-run anytime after you've downloaded the Transcript files locally.  This is just example code.  You might want to use your cloud storage provider of choice if you want to scale this for full history and daily / intraday scriptted runs.

In [7]:
# Function to recursively flatten XML into a dictionary, avoiding the need to hardcode XPaths
def flatten_xml(element, parent_key='', sep='_'):
    items = []
    for child in element:
        key = f"{parent_key}{sep}{child.tag}" if parent_key else child.tag
        if len(child):  # If the child has children, recursively flatten
            items.extend(flatten_xml(child, key, sep=sep).items())
        else:
            items.append((key, child.text.strip() if child.text else None))
    # Add attributes of the current element
    for attr_key, attr_value in element.attrib.items():
        items.append((f"{parent_key}{sep}{attr_key}" if parent_key else attr_key, attr_value))
    return dict(items)

In [8]:
# Get the list of files in the local directory
xml_files = [f for f in os.listdir(local_directory) if f.endswith('.xml')]

# Initialize an empty list to store flattened data
all_flattened_data = []

# Iterate through all XML files
for xml_file in xml_files:
    file_path = os.path.join(local_directory, xml_file)
    
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    # Flatten the XML and append to the list
    flattened_data = flatten_xml(root)
    all_flattened_data.append(flattened_data)

# Convert the list of dictionaries into a pandas DataFrame
df_all_flattened = pd.DataFrame(all_flattened_data)

df_all_flattened.head()

,EventStory_Headline,EventStory_Body,EventStory_Id,EventStory_expirationDate,EventStory_action,EventStory_storyType,EventStory_version,eventTitle,city,companyName,companyTicker,startDate,companyId,CUSIP,SEDOL,ISIN,Id,lastUpdate,eventTypeId,eventTypeName,EventStory
0,Edited Transcript of DFS.N^E25 earnings confer...,Q1 2018 Discover Financial Services Earnings C...,11470119.F,"Thursday, April 26, 2018 at 9:00:00pm GMT",publish,transcript,Final,Q1 2018 Discover Financial Services Earnings Call,MCLEAN,Discover Financial Services,DFS.N^E25,26-Apr-18 9:00pm GMT,204177,None,None,None,11470119,"Friday, June 6, 2025 at 3:09:36am GMT",1,Earning Conference Call/Presentation,NaN
1,Edited Transcript of S32.AX M&A conference cal...,South32 Ltd to Acquire Arizona Mining in Agree...,11561039.F,"Sunday, June 17, 2018 at 11:30:00pm GMT",publish,transcript,Final,South32 Ltd to Acquire Arizona Mining in Agree...,VANCOUVER,South32 Ltd,S32.AX,17-Jun-18 11:30pm GMT,276982,None,BWSW5D9,AU000000S320,11561039,"Friday, June 6, 2025 at 2:59:20am GMT",34,M&A Conference Call/Presentation,NaN
2,Edited Transcript of BAVA.CO earnings conferen...,Q3 2019 Bavarian Nordic A/S Earnings Call\n\nK...,12827487.F,"Thursday, November 7, 2019 at 1:00:00pm GMT",publish,transcript,Final,Q3 2019 Bavarian Nordic A/S Earnings Call,Kvistgaard,Bavarian Nordic A/S,BAVA.CO,7-Nov-19 1:00pm GMT,156334,None,5564619,DK0015998017,12827487,"Monday, June 9, 2025 at 7:29:12am GMT",1,Earning Conference Call/Presentation,NaN
3,Edited Transcript of DFS.N^E25 earnings confer...,Q1 2021 Discover Financial Services Earnings C...,14645148.F,"Thursday, April 22, 2021 at 12:00:00pm GMT",publish,transcript,Final,Q1 2021 Discover Financial Services Earnings Call,MCLEAN,Discover Financial Services,DFS.N^E25,22-Apr-21 12:00pm GMT,204177,None,None,None,14645148,"Friday, June 6, 2025 at 3:25:07am GMT",1,Earning Conference Call/Presentation,NaN
4,Edited Transcript of TCNS.NS^I24 earnings conf...,Q1 2022 TCNS Clothing Co Ltd Earnings Call\n\n...,14949356.F,"Friday, August 13, 2021 at 12:30:00pm GMT",publish,transcript,Final,Q1 2022 TCNS Clothing Co Ltd Earnings Call,MUMBAI,TCNS Clothing Co Ltd,TCNS.NS^I24,13-Aug-21 12:30pm GMT,322057,None,None,None,14949356,"Friday, June 6, 2025 at 3:57:59pm GMT",1,Earning Conference Call/Presentation,NaN


### Filter universe to be processed via GenAI

The only_US parameter will focus on just U.S. based companies, since this experiment was created around the time of the all President Trump tariff concerns.  The max_companies will throttle how many companies you pass through OpenAI.

In [9]:
transcripts_df = df_all_flattened

# Filter only US-based companies
if only_US:
    transcripts_df = transcripts_df[transcripts_df['companyTicker'].fillna('').str.endswith(('N', 'OQ'))]

# Cap max companeis to be processed by GenAI
transcripts_df = transcripts_df.head(max_companies)

transcripts_df.head()

,EventStory_Headline,EventStory_Body,EventStory_Id,EventStory_expirationDate,EventStory_action,EventStory_storyType,EventStory_version,eventTitle,city,companyName,companyTicker,startDate,companyId,CUSIP,SEDOL,ISIN,Id,lastUpdate,eventTypeId,eventTypeName,EventStory
19,Edited Transcript of MDT.N earnings conference...,Q4 2025 Medtronic PLC Earnings Call\n\nDublin ...,15877481.F,"Wednesday, May 21, 2025 at 12:00:00pm GMT",publish,transcript,Final,Q4 2025 Medtronic PLC Earnings Call,Dublin,Medtronic PLC,MDT.N,21-May-25 12:00pm GMT,277190,None,BTN1Y11,IE00BTN1Y115,15877481,"Friday, June 6, 2025 at 11:05:28pm GMT",1,Earning Conference Call/Presentation,NaN
31,Edited Transcript of RBLX.N earnings conferenc...,"Q2 2024 Roblox Corp Earnings Call\n\n Jun 6, 2...",16048862.F,"Thursday, August 1, 2024 at 12:30:00pm GMT",publish,transcript,Final,Q2 2024 Roblox Corp Earnings Call,,Roblox Corp (Pre-Reincorporation),RBLX.N,1-Aug-24 12:30pm GMT,399939,771049103,BMWBC20,US7710491033,16048862,"Friday, June 6, 2025 at 3:03:36am GMT",1,Earning Conference Call/Presentation,NaN
41,Edited Transcript of RBLX.N corporate analyst ...,Roblox Corp Roblox Developers Conference\n\n J...,16107822.F,"Friday, September 6, 2024 at 5:00:00pm GMT",publish,transcript,Final,Roblox Corp Roblox Developers Conference,,Roblox Corp (Pre-Reincorporation),RBLX.N,6-Sep-24 5:00pm GMT,399939,771049103,BMWBC20,US7710491033,16107822,"Friday, June 6, 2025 at 3:05:52am GMT",31,Analyst Meeting,NaN
66,Edited Transcript of MBI.N earnings conference...,Q4 2024 MBIA Inc Earnings Call \n\nARMONK Jun ...,16196531.F,"Friday, February 28, 2025 at 1:00:00pm GMT",publish,transcript,Final,Q4 2024 MBIA Inc Earnings Call,ARMONK,MBIA Inc,MBI.N,28-Feb-25 1:00pm GMT,88095,55262C100,2547044,US55262C1009,16196531,"Friday, June 6, 2025 at 5:12:19pm GMT",1,Earning Conference Call/Presentation,NaN
77,Edited Transcript of WDFC.OQ earnings conferen...,Q1 2025 WD-40 Co Earnings Call\n\nSAN DIEGO Ju...,16208622.F,"Friday, January 10, 2025 at 10:00:00pm GMT",publish,transcript,Final,Q1 2025 WD-40 Co Earnings Call,SAN DIEGO,WD-40 Co,WDFC.OQ,10-Jan-25 10:00pm GMT,104082,929236107,2944742,US9292361071,16208622,"Friday, June 6, 2025 at 5:16:05pm GMT",1,Earning Conference Call/Presentation,NaN


### Parse out Country Supplier % from Transcripts with GenAI

This is where the GenAI magic happens.  OpenAI is asked to return the informaiton in JSON format, for easier structured parsing.

#### With meta-prompting to OpenAI, I used the prompt below to generate the more elaborate prompt seen in the system_message.
"Extract the percentage of goods sourced from each country, discussed in the user provided transcript of a company's earning call.  Extract an exact quote from the text which contains both the supplier country and the percentage of goods supplied.  Return the extraction results in array of JSON elements named "sources" containing the elements named COUNTRY, SOURCING_PERCENT, QUOTE.  Do not parse out financial percentages unrelated to sourcing.  Do not hallucinate.  Do not make up quotes.  Do not make up percentages which are not mentioned in the quotes."

In [10]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key = ai_key,  
    api_version = ai_version,
    azure_endpoint = ai_endpoint
    )

In [11]:
system_message = """Extract the percentage of goods sourced from each country discussed in the user-provided transcript of a company's earning call. Ensure that the extraction includes an exact quote from the transcript that explicitly mentions both the supplier country and the percentage of goods supplied.

- Do not parse out financial percentages unrelated to sourcing (e.g., revenue growth, profit margins).
- Do not fabricate information, quotes, or percentages that are not explicitly mentioned in the text.
- If no relevant sourcing data is found, return an empty array.
- Provide results in the specified JSON format.

# Steps

1. Identify passages in the transcript explicitly discussing the sourcing of goods and containing both a country and a percentage reference.
2. Extract the relevant country, the percentage of goods sourced, and the exact quote including both the country and percentage.
3. Ensure that only sourcing percentages are considered—exclude unrelated financial percentages.
4. Compile the results into an array of JSON objects with the keys `COUNTRY`, `SOURCING_PERCENT`, and `QUOTE`.

# Output Format

The output should be an array of JSON objects named `sources`, with each object containing:
- `COUNTRY`: The name of the supplier country.
- `SOURCING_PERCENT`: The percentage of goods sourced from the country, as stated in the transcript.
- `QUOTE`: The exact quote from the text containing both the country and the percentage.

If no relevant data is found, return:
```json
"sources": []
```

# Example

Given the input transcript:

> "Approximately 45% of our goods are sourced from China, while 20% come from Vietnam and 10% from India. Additionally, we have started sourcing 5% from the Philippines as part of our diversification strategy."

The output should be:

```json
{
  "sources": [
    {
      "COUNTRY": "China",
      "SOURCING_PERCENT": "45%",
      "QUOTE": "Approximately 45% of our goods are sourced from China."
    },
    {
      "COUNTRY": "Vietnam",
      "SOURCING_PERCENT": "20%",
      "QUOTE": "20% come from Vietnam."
    },
    {
      "COUNTRY": "India",
      "SOURCING_PERCENT": "10%",
      "QUOTE": "10% from India."
    },
    {
      "COUNTRY": "Philippines",
      "SOURCING_PERCENT": "5%",
      "QUOTE": "We have started sourcing 5% from the Philippines."
    }
  ]
}
```

# Notes

- Ensure quotes retain their original wording and context.
- Skip lines or phrases without specific sourcing percentages.
- Avoid including unrelated financial data or synthetic examples.
"""

In [12]:
# Function to extract sourcing information from a long transcript using OpenAI
def extract_from_transcript(long_transcript_text):
    
    # Don't waste time if looking up if there is no Transcript
    if long_transcript_text is None:
        return (0, '{ "sources": [] }', 0)

    supplies_messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': long_transcript_text}
    ]

    attempts = 0
    sources = '{ "sources": [] }'
    prompt_tokens = 0

    # Retry if no response or empty response, since sometimes OpenAI needs to be asked a few times to get it right.
    while attempts < max_attempts:
        attempts += 1

        response = client.chat.completions.create(
            model=ai_model,
            messages=supplies_messages,
            response_format={"type": "json_object"},
            n=1
        )
        sources = response.choices[0].message.content
        prompt_tokens = response.usage.prompt_tokens

        # If we have an actual response, make no more attempts
        sources_json = json.loads(sources)
        if 'sources' in sources_json and len(sources_json['sources']) > 0:
            break        

    return (prompt_tokens, sources, attempts)

In [13]:
%%time

# Apply the extract_from_transcript function to each EventStory_Body in the transcripts_df
transcripts_df[['prompt_tokens', 'sources', 'attempts']] = transcripts_df['EventStory_Body'].apply(
    lambda x: pd.Series(extract_from_transcript(x))
)

transcripts_df.head()

CPU times: total: 2.67 s
Wall time: 30min 24s


,EventStory_Headline,EventStory_Body,EventStory_Id,EventStory_expirationDate,EventStory_action,EventStory_storyType,EventStory_version,eventTitle,city,companyName,companyTicker,startDate,companyId,CUSIP,SEDOL,ISIN,Id,lastUpdate,eventTypeId,eventTypeName,EventStory,prompt_tokens,sources,attempts
19,Edited Transcript of MDT.N earnings conference...,Q4 2025 Medtronic PLC Earnings Call\n\nDublin ...,15877481.F,"Wednesday, May 21, 2025 at 12:00:00pm GMT",publish,transcript,Final,Q4 2025 Medtronic PLC Earnings Call,Dublin,Medtronic PLC,MDT.N,21-May-25 12:00pm GMT,277190,None,BTN1Y11,IE00BTN1Y115,15877481,"Friday, June 6, 2025 at 11:05:28pm GMT",1,Earning Conference Call/Presentation,NaN,17363,"{\n ""sources"": []\n}",1
31,Edited Transcript of RBLX.N earnings conferenc...,"Q2 2024 Roblox Corp Earnings Call\n\n Jun 6, 2...",16048862.F,"Thursday, August 1, 2024 at 12:30:00pm GMT",publish,transcript,Final,Q2 2024 Roblox Corp Earnings Call,,Roblox Corp (Pre-Reincorporation),RBLX.N,1-Aug-24 12:30pm GMT,399939,771049103,BMWBC20,US7710491033,16048862,"Friday, June 6, 2025 at 3:03:36am GMT",1,Earning Conference Call/Presentation,NaN,10054,"{\n ""sources"": []\n}",1
41,Edited Transcript of RBLX.N corporate analyst ...,Roblox Corp Roblox Developers Conference\n\n J...,16107822.F,"Friday, September 6, 2024 at 5:00:00pm GMT",publish,transcript,Final,Roblox Corp Roblox Developers Conference,,Roblox Corp (Pre-Reincorporation),RBLX.N,6-Sep-24 5:00pm GMT,399939,771049103,BMWBC20,US7710491033,16107822,"Friday, June 6, 2025 at 3:05:52am GMT",31,Analyst Meeting,NaN,36150,"{\n ""sources"": []\n}",1
66,Edited Transcript of MBI.N earnings conference...,Q4 2024 MBIA Inc Earnings Call \n\nARMONK Jun ...,16196531.F,"Friday, February 28, 2025 at 1:00:00pm GMT",publish,transcript,Final,Q4 2024 MBIA Inc Earnings Call,ARMONK,MBIA Inc,MBI.N,28-Feb-25 1:00pm GMT,88095,55262C100,2547044,US55262C1009,16196531,"Friday, June 6, 2025 at 5:12:19pm GMT",1,Earning Conference Call/Presentation,NaN,4542,"{\n ""sources"": []\n}",1
77,Edited Transcript of WDFC.OQ earnings conferen...,Q1 2025 WD-40 Co Earnings Call\n\nSAN DIEGO Ju...,16208622.F,"Friday, January 10, 2025 at 10:00:00pm GMT",publish,transcript,Final,Q1 2025 WD-40 Co Earnings Call,SAN DIEGO,WD-40 Co,WDFC.OQ,10-Jan-25 10:00pm GMT,104082,929236107,2944742,US9292361071,16208622,"Friday, June 6, 2025 at 5:16:05pm GMT",1,Earning Conference Call/Presentation,NaN,9671,"{\n ""sources"": []\n}",1


### Format the GenAI output into separate columns

The Excel format with the Transcript data and the GenAI parsed data will help non-technical users better leverage this information.

In [14]:
# Function to expand the extract JSON into multiple rows
def expand_sources(row):
    sources = json.loads(row['sources'])
    if 'sources' in sources and len(sources['sources']) > 0:
        expanded_rows = []
        for source in sources['sources']:
            new_row = row.copy()
            new_row['COUNTRY'] = source['COUNTRY']
            new_row['SOURCING_PERCENT'] = source['SOURCING_PERCENT']
            new_row['QUOTE'] = source['QUOTE']
            expanded_rows.append(new_row)
        return pd.DataFrame(expanded_rows)
    else:
        row['COUNTRY'] = None
        row['SOURCING_PERCENT'] = None
        row['QUOTE'] = None
        return pd.DataFrame([row])

# Apply the function to each row in the dataframe and concatenate the results
expanded_transcripts_df = pd.concat(transcripts_df.apply(expand_sources, axis=1).to_list(), ignore_index=True)

expanded_transcripts_df.head()

,EventStory_Headline,EventStory_Body,EventStory_Id,EventStory_expirationDate,EventStory_action,EventStory_storyType,EventStory_version,eventTitle,city,companyName,companyTicker,startDate,companyId,CUSIP,SEDOL,ISIN,Id,lastUpdate,eventTypeId,eventTypeName,EventStory,prompt_tokens,sources,attempts,COUNTRY,SOURCING_PERCENT,QUOTE
0,Edited Transcript of MDT.N earnings conference...,Q4 2025 Medtronic PLC Earnings Call\n\nDublin ...,15877481.F,"Wednesday, May 21, 2025 at 12:00:00pm GMT",publish,transcript,Final,Q4 2025 Medtronic PLC Earnings Call,Dublin,Medtronic PLC,MDT.N,21-May-25 12:00pm GMT,277190,None,BTN1Y11,IE00BTN1Y115,15877481,"Friday, June 6, 2025 at 11:05:28pm GMT",1,Earning Conference Call/Presentation,NaN,17363,"{\n ""sources"": []\n}",1,None,None,None
1,Edited Transcript of RBLX.N earnings conferenc...,"Q2 2024 Roblox Corp Earnings Call\n\n Jun 6, 2...",16048862.F,"Thursday, August 1, 2024 at 12:30:00pm GMT",publish,transcript,Final,Q2 2024 Roblox Corp Earnings Call,,Roblox Corp (Pre-Reincorporation),RBLX.N,1-Aug-24 12:30pm GMT,399939,771049103,BMWBC20,US7710491033,16048862,"Friday, June 6, 2025 at 3:03:36am GMT",1,Earning Conference Call/Presentation,NaN,10054,"{\n ""sources"": []\n}",1,None,None,None
2,Edited Transcript of RBLX.N corporate analyst ...,Roblox Corp Roblox Developers Conference\n\n J...,16107822.F,"Friday, September 6, 2024 at 5:00:00pm GMT",publish,transcript,Final,Roblox Corp Roblox Developers Conference,,Roblox Corp (Pre-Reincorporation),RBLX.N,6-Sep-24 5:00pm GMT,399939,771049103,BMWBC20,US7710491033,16107822,"Friday, June 6, 2025 at 3:05:52am GMT",31,Analyst Meeting,NaN,36150,"{\n ""sources"": []\n}",1,None,None,None
3,Edited Transcript of MBI.N earnings conference...,Q4 2024 MBIA Inc Earnings Call \n\nARMONK Jun ...,16196531.F,"Friday, February 28, 2025 at 1:00:00pm GMT",publish,transcript,Final,Q4 2024 MBIA Inc Earnings Call,ARMONK,MBIA Inc,MBI.N,28-Feb-25 1:00pm GMT,88095,55262C100,2547044,US55262C1009,16196531,"Friday, June 6, 2025 at 5:12:19pm GMT",1,Earning Conference Call/Presentation,NaN,4542,"{\n ""sources"": []\n}",1,None,None,None
4,Edited Transcript of WDFC.OQ earnings conferen...,Q1 2025 WD-40 Co Earnings Call\n\nSAN DIEGO Ju...,16208622.F,"Friday, January 10, 2025 at 10:00:00pm GMT",publish,transcript,Final,Q1 2025 WD-40 Co Earnings Call,SAN DIEGO,WD-40 Co,WDFC.OQ,10-Jan-25 10:00pm GMT,104082,929236107,2944742,US9292361071,16208622,"Friday, June 6, 2025 at 5:16:05pm GMT",1,Earning Conference Call/Presentation,NaN,9671,"{\n ""sources"": []\n}",1,None,None,None


#### Verify that the Quote from GenAI actually exists in the Transcript

Hallucinations can still happen with GenAI, even if you ask nicely to avoid hallucinations.  This code will check that the supporting QUOTE from GenAI actually exists in the original Transcript

In [15]:
# Function to normalize text by removing unwanted characters and extra spaces
# Sometimes GenAI does weird stuff like changing a ' to a ’ which causing basic text matching to fail.
# We only care about matching letters, numbers and the % sign.
def normalize_text(text):
    # Remove all characters except alphanumeric and %
    text = re.sub(r'[^a-zA-Z0-9%]', ' ', text)
    # Convert to lowercase and remove extra spaces
    return re.sub(r'\s+', ' ', text).strip().lower()

def check_quote_in_transcript(row):
    if pd.isna(row['QUOTE']) or row['QUOTE'] == '':
        return 0
    normalized_quote = normalize_text(row['QUOTE'])
    normalized_transcript = normalize_text(row['EventStory_Body'])
    if normalized_quote in normalized_transcript:
        # Find the paragraph containing the quote
        paragraphs = row['EventStory_Body'].split('\r\n')
        for paragraph in paragraphs:
            if normalized_quote in normalize_text(paragraph):
                return 1
    return 0

expanded_transcripts_df[['quote_in_transcript']] = expanded_transcripts_df.apply(
    lambda row: pd.Series(check_quote_in_transcript(row)), axis=1
)

### Write to Excel

Success!  Now time to write the output and share with your stock analyst colleagues.

In [16]:
# Create the local directory if it doesn't exist
os.makedirs(genai_output_dir, exist_ok=True)

# Get the current date and time
current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Define the file name with the timestamp
extract_file_name = f"{genai_output_dir}/supplier_countries_{current_timestamp}.xlsx"

# Write the dataframe to an Excel file
expanded_transcripts_df.to_excel(extract_file_name, index=False)